In [1]:
import numpy as np
import pandas as pd

print("Hello world")

Hello world


In [ ]:
data = open("liiklusjarelevalve_3.csv", encoding = "UTF-8")

# -----------------------------------

n = 0
data_list = []

for row in data:
    x = row.split("\t")
    
    l = len(x)
    k = 0

    while k < l:
        x[k] = x[k].replace('"', '')
        k += 1
        
    if n == 0: # First row of the csv file is the variables list so it is separated from the others
        cols = x
        n += 1
        
    else:
        data_list.append(x)
        n += 1
        if n == 10:
            break
            
# ------------------------------------

traffic_df = pd.DataFrame(data_list, columns = cols)

traffic_df

In [ ]:
traffic_df = traffic_df.rename(columns = {"SyyteoLiik\n" : "SyyteoLiik"})